In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
filepath = '/content/gdrive/My Drive/tabular_playground/'
train = pd.read_csv(filepath+'train.csv')
test=pd.read_csv(filepath+'test.csv')

Mounted at /content/gdrive


In [2]:
cat_cols = [feature for feature in train.columns if 'cat' in feature]  #cat으로 시작하는 열 이름 모으기
num_cols=[num for num in train.columns if 'cont' in num]    #cont로 시작하는 열 이름 모으기
print(cat_cols)
print(num_cols)

['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']


In [3]:
# cat에 있던 알파벳 -> 숫자로 변환
from sklearn.preprocessing import LabelEncoder
for i in cat_cols:
  le = LabelEncoder()
  train[i] = le.fit_transform(train[i])
  test[i]=le.fit_transform(test[i])

In [4]:
# 인코딩 시킨 데이터들을 train과 test 셋으로 다시 나눈다.
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
y_train = train['target']
X_train = train.drop('target', axis=1)
X_test = test

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state = 0,shuffle=True)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
# 1. hidden unit을 점차 줄여가는 형식으로 해봄.
activ_ = 'relu'
h_units = 36
n_inputs=24

def dnn_model():
  model=Sequential()

  #hidden layer #1
  model.add(Dense(units = (h_units+300), input_dim = n_inputs))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #2
  model.add(Dense(units = (h_units+200)))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #3
  model.add(Dense(units = (h_units+150)))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #4
  model.add(Dense(units = (h_units+100)))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #5
  model.add(Dense(units = (h_units+50)))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #6
  model.add(Dense(units = (h_units+25)))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #7
  model.add(Dense(units = h_units))
  model.add(Activation(activ_))

  #output layer
  model.add(Dense(units = 1))

  return model


In [8]:
model =dnn_model()
model.summary()
opti = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer = opti,loss = 'mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 336)               8400      
_________________________________________________________________
batch_normalization (BatchNo (None, 336)               1344      
_________________________________________________________________
activation (Activation)      (None, 336)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 236)               79532     
_________________________________________________________________
batch_normalization_1 (Batch (None, 236)               944       
_________________________________________________________________
activation_1 (Activation)    (None, 236)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 186)               4

In [ ]:
hist = model.fit(X_train, y_train, epochs=100, verbose=True,batch_size=32, validation_data=(X_valid,y_valid),shuffle=True)

Epoch 1/100
6563/6563 [==============================] - 39s 6ms/step - loss: 0.8025 - mean_squared_error: 0.8025 - val_loss: 0.7923 - val_mean_squared_error: 0.7923
Epoch 2/100
6563/6563 [==============================] - 39s 6ms/step - loss: 0.7715 - mean_squared_error: 0.7715 - val_loss: 0.7636 - val_mean_squared_error: 0.7636
Epoch 3/100
6563/6563 [==============================] - 39s 6ms/step - loss: 0.7609 - mean_squared_error: 0.7609 - val_loss: 0.7493 - val_mean_squared_error: 0.7493
Epoch 4/100
6563/6563 [==============================] - 39s 6ms/step - loss: 0.7575 - mean_squared_error: 0.7575 - val_loss: 0.7475 - val_mean_squared_error: 0.7475
Epoch 5/100
6563/6563 [==============================] - 38s 6ms/step - loss: 0.7557 - mean_squared_error: 0.7557 - val_loss: 0.7508 - val_mean_squared_error: 0.7508
Epoch 6/100
6563/6563 [==============================] - 38s 6ms/step - loss: 0.7541 - mean_squared_error: 0.7541 - val_loss: 0.7452 - val_mean_squared_error: 0.7452
Epoc

In [ ]:
y_pred = model.predict(X_valid,batch_size = 64)
print(y_pred.shape,y_valid.shape)
model.evaluate(x=X_valid,y=y_valid,batch_size=64,verbose=True)

(90000, 1) (90000,)
1407/1407 [==============================] - 3s 2ms/step - loss: 0.7424 - mean_squared_error: 0.7424


[0.7424416542053223, 0.7424416542053223]

In [ ]:
sub = pd.read_csv(filepath+'sample_submission.csv')
y_pred = model.predict(X_test,batch_size=32)
sub['target'] = y_pred
sub.to_csv(filepath+'deep_1.csv', index =False)
sub.head()

,id,target
0,0,7.481535
1,5,7.865315
2,15,7.617304
3,16,7.410262
4,17,7.168121


In [9]:
# 2. 유닛을 똑같이 줌
activ_ = 'relu'
h_units = 36
n_inputs=24
def dnn_model():
  model=Sequential()
  #hidden layer #1
  model.add(Dense(units = h_units , input_dim = n_inputs))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #2
  model.add(Dense(units = h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #3
  model.add(Dense(units =h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #4
  model.add(Dense(units =h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #5
  model.add(Dense(units = h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #6
  model.add(Dense(units =h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))

  #hidden layer #7
  model.add(Dense(units = h_units))
  model.add(Activation(activ_))

  #output layer
  model.add(Dense(units = 1))

  return model

In [10]:
model =dnn_model()
model.summary()
opti = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer = opti,loss = 'mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 36)                900       
_________________________________________________________________
batch_normalization_6 (Batch (None, 36)                144       
_________________________________________________________________
activation_7 (Activation)    (None, 36)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 36)                1332      
_________________________________________________________________
batch_normalization_7 (Batch (None, 36)                144       
_________________________________________________________________
activation_8 (Activation)    (None, 36)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 36)               

In [ ]:
hist = model.fit(X_train, y_train, epochs=100, verbose=True,batch_size=64, validation_data=(X_valid,y_valid),shuffle=True)

Epoch 1/100
3282/3282 [==============================] - 20s 6ms/step - loss: 1.5325 - mean_squared_error: 1.5325 - val_loss: 0.8149 - val_mean_squared_error: 0.8149
Epoch 2/100
3282/3282 [==============================] - 19s 6ms/step - loss: 0.7910 - mean_squared_error: 0.7910 - val_loss: 0.8093 - val_mean_squared_error: 0.8093
Epoch 3/100
3282/3282 [==============================] - 19s 6ms/step - loss: 0.7720 - mean_squared_error: 0.7720 - val_loss: 0.7546 - val_mean_squared_error: 0.7546
Epoch 4/100
3282/3282 [==============================] - 19s 6ms/step - loss: 0.7618 - mean_squared_error: 0.7618 - val_loss: 0.7684 - val_mean_squared_error: 0.7684
Epoch 5/100
3282/3282 [==============================] - 19s 6ms/step - loss: 0.7579 - mean_squared_error: 0.7579 - val_loss: 0.7650 - val_mean_squared_error: 0.7650
Epoch 6/100
3282/3282 [==============================] - 19s 6ms/step - loss: 0.7573 - mean_squared_error: 0.7573 - val_loss: 0.7476 - val_mean_squared_error: 0.7476
Epoc

In [ ]:
y_pred = model.predict(X_valid,batch_size = 64)
print(y_pred.shape,y_valid.shape)
model.evaluate(x=X_valid,y=y_valid,batch_size=64,verbose=True)

(90000, 1) (90000,)
1407/1407 [==============================] - 3s 2ms/step - loss: 0.7405 - mean_squared_error: 0.7405


[0.7404606938362122, 0.7404606938362122]

In [16]:
# 유닛을 32개 더 줌
activ_ = 'relu'
h_units = 64
n_inputs=24

def dnn_model():
  model=Sequential()

  #hidden layer #1
  model.add(Dense(units = h_units , input_dim = n_inputs))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #2
  model.add(Dense(units = h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))
  #hidden layer #3
  model.add(Dense(units =h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))

  #hidden layer #4
  model.add(Dense(units =h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))

    #hidden layer #5
  model.add(Dense(units = h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))

  #hidden layer #6
  model.add(Dense(units =h_units ))
  model.add(BatchNormalization())
  model.add(Activation(activ_))

  #hidden layer #7
  model.add(Dense(units = h_units))
  model.add(Activation(activ_))

  #output layer
  model.add(Dense(units = 1))

  return model

In [17]:
model =dnn_model()
model.summary()
opti = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer = opti,loss = 'mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 64)                1600      
_________________________________________________________________
batch_normalization_24 (Batc (None, 64)                256       
_________________________________________________________________
activation_28 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 64)                4160      
_________________________________________________________________
batch_normalization_25 (Batc (None, 64)                256       
_________________________________________________________________
activation_29 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)               

In [18]:
hist = model.fit(X_train, y_train, epochs=100, verbose=True,batch_size=64, validation_data=(X_valid,y_valid),shuffle=True)

Epoch 1/100
3282/3282 [==============================] - 10s 3ms/step - loss: 1.2777 - mean_squared_error: 1.2777 - val_loss: 0.8847 - val_mean_squared_error: 0.8847
Epoch 2/100
3282/3282 [==============================] - 8s 3ms/step - loss: 0.7953 - mean_squared_error: 0.7953 - val_loss: 0.7535 - val_mean_squared_error: 0.7535
Epoch 3/100
3282/3282 [==============================] - 8s 3ms/step - loss: 0.7681 - mean_squared_error: 0.7681 - val_loss: 0.7539 - val_mean_squared_error: 0.7539
Epoch 4/100
3282/3282 [==============================] - 8s 3ms/step - loss: 0.7618 - mean_squared_error: 0.7618 - val_loss: 0.7536 - val_mean_squared_error: 0.7536
Epoch 5/100
3282/3282 [==============================] - 8s 3ms/step - loss: 0.7539 - mean_squared_error: 0.7539 - val_loss: 0.7469 - val_mean_squared_error: 0.7469
Epoch 6/100
3282/3282 [==============================] - 9s 3ms/step - loss: 0.7510 - mean_squared_error: 0.7510 - val_loss: 0.7554 - val_mean_squared_error: 0.7554
Epoch 7/1

In [19]:
y_pred = model.predict(X_valid,batch_size = 64)
print(y_pred.shape,y_valid.shape)
model.evaluate(x=X_valid,y=y_valid,batch_size=64,verbose=True)

(90000, 1) (90000,)
1407/1407 [==============================] - 1s 867us/step - loss: 0.7372 - mean_squared_error: 0.7372


[0.7372406125068665, 0.7372406125068665]

In [20]:
sub = pd.read_csv(filepath+'sample_submission.csv')
y_pred = model.predict(X_test,batch_size=32)
sub['target'] = y_pred
sub.to_csv(filepath+'deep_learning.csv', index =False)
sub.head()

,id,target
0,0,7.481429
1,5,7.866292
2,15,7.582352
3,16,7.337811
4,17,7.118035
